In [1]:
import pandas as pd
import pdb
import numpy as np
import matplotlib.pyplot as plt
import analytics_functions as analytics
import seaborn as sns
from sklearn.decomposition import PCA
import ast

%matplotlib inline 
from ipywidgets import *

Load in and display the dataframe

In [2]:
df = pd.read_csv('Data/playlists_18_03_19.csv')
df

,track name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,type,id,uri,track_href,analysis_url,duration_ms,time_signature,artists,genres,playlist
0,Just You and I,0.567,0.696,9,-5.416,1,0.3010,0.456000,0.000000,0.1130,...,audio_features,2n48BjaFSczfeIG3HaIaPv,spotify:track:2n48BjaFSczfeIG3HaIaPv,https://api.spotify.com/v1/tracks/2n48BjaFSczf...,https://api.spotify.com/v1/audio-analysis/2n48...,174983,4,['Tom Walker'],"['indie anthem-folk', 'modern rock']",Hot Hits UK
1,Don't Call Me Up,0.674,0.881,9,-2.853,1,0.1470,0.296000,0.000003,0.0793,...,audio_features,5WHTFyqSii0lmT9R21abT8,spotify:track:5WHTFyqSii0lmT9R21abT8,https://api.spotify.com/v1/tracks/5WHTFyqSii0l...,https://api.spotify.com/v1/audio-analysis/5WHT...,178480,4,['Mabel'],"['dance pop', 'house', 'pop', 'post-teen pop',...",This Is How We Do
2,One Drink,0.707,0.771,11,-4.371,1,0.0821,0.034200,0.000002,0.0684,...,audio_features,7vSm2hckpPHTrIHpqmqOze,spotify:track:7vSm2hckpPHTrIHpqmqOze,https://api.spotify.com/v1/tracks/7vSm2hckpPHT...,https://api.spotify.com/v1/audio-analysis/7vSm...,200720,4,['Picture This'],['indie anthem-folk'],Hot Hits UK
3,Dancing With A Stranger (with Normani),0.741,0.520,8,-7.513,1,0.0656,0.450000,0.000002,0.2220,...,audio_features,6Qs4SXO9dwPj5GKvVOv8Ki,spotify:track:6Qs4SXO9dwPj5GKvVOv8Ki,https://api.spotify.com/v1/tracks/6Qs4SXO9dwPj...,https://api.spotify.com/v1/audio-analysis/6Qs4...,171030,4,"['Sam Smith', 'Normani']","[['pop'], ['dance pop', 'pop', 'post-teen pop'...",Hot Hits UK
4,Hello My Love,0.679,0.717,0,-4.548,1,0.0384,0.097100,0.000000,0.1240,...,audio_features,4EtIza2cKMmR2XP7idebXG,spotify:track:4EtIza2cKMmR2XP7idebXG,https://api.spotify.com/v1/tracks/4EtIza2cKMmR...,https://api.spotify.com/v1/audio-analysis/4EtI...,214212,4,['Westlife'],"['boy band', 'dance pop', 'europop']",Hot Hits UK
5,Someone You Loved,0.501,0.405,1,-5.679,1,0.0319,0.751000,0.000000,0.1050,...,audio_features,2TIlqbIneP0ZY1O0EzYLlc,spotify:track:2TIlqbIneP0ZY1O0EzYLlc,https://api.spotify.com/v1/tracks/2TIlqbIneP0Z...,https://api.spotify.com/v1/audio-analysis/2TIl...,182161,4,['Lewis Capaldi'],"['pop', 'uk pop']",Hot Hits UK
6,Black,0.495,0.757,7,-7.150,1,0.4220,0.359000,0.000000,0.1160,...,audio_features,2IzjEAXzEYu5jYpAbiRxTk,spotify:track:2IzjEAXzEYu5jYpAbiRxTk,https://api.spotify.com/v1/tracks/2IzjEAXzEYu5...,https://api.spotify.com/v1/audio-analysis/2Izj...,228280,4,['Dave'],"['indie r&b', 'uk hip hop']",Rap UK
7,Giant (with Rag'n'Bone Man),0.807,0.887,1,-4.311,0,0.0361,0.016000,0.000503,0.0811,...,audio_features,5itOtNx0WxtJmi1TQ3RuRd,spotify:track:5itOtNx0WxtJmi1TQ3RuRd,https://api.spotify.com/v1/tracks/5itOtNx0WxtJ...,https://api.spotify.com/v1/audio-analysis/5itO...,229184,4,"['Calvin Harris', ""Rag'n'Bone Man""]","[['edm', 'electro house', 'house', 'pop', 'pro...",Massive Dance Hits
8,"break up with your girlfriend, i'm bored",0.726,0.554,5,-5.290,0,0.0917,0.042100,0.000000,0.1060,...,audio_features,4kV4N9D1iKVxx1KLvtTpjS,spotify:track:4kV4N9D1iKVxx1KLvtTpjS,https://api.spotify.com/v1/tracks/4kV4N9D1iKVx...,https://api.spotify.com/v1/audio-analysis/4kV4...,190440,4,['Ariana Grande'],"['dance pop', 'pop', 'post-teen pop']",This Is How We Do
9,i'm so tired...,0.599,0.733,11,-7.058,1,0.2030,0.176000,0.000000,0.2420,...,audio_features,7COXchtUOMd6uIT6HvmRaI,spotify:track:7COXchtUOMd6uIT6HvmRaI,https://api.spotify.com/v1/tracks/7COXchtUOMd6...,https://api.spotify.com/v1/audio-analysis/7COX...,162583,4,"['Lauv', 'Troye Sivan']","[['dance pop', 'pop', 'post-teen pop'], ['danc...",Hot Hits UK


In [3]:
data_dict = analytics.data_frame_to_dict(df)

/home/matt/Dropbox/spotiplaylist_explorer/analytics_functions.py:33: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  matrix = df.as_matrix()


# Perform PCA to reduce the set of feature to 2 dimensions

In [4]:
column_headers , feature_vecs = analytics.dict_to_feature_vectors(data_dict)

feature_vecs = np.transpose(feature_vecs)
pca = PCA(n_components=2)
X_embedded = pca.fit_transform(feature_vecs)
x = np.transpose(X_embedded)[0]
y = np.transpose(X_embedded)[1]
data_dict["PCA_X"] = x
data_dict["PCA_Y"] = y

#Plotting in seaborn requires the values to be converted to numpy.float64/int.64:
for key in data_dict.keys() :
    if isinstance(data_dict[key][0],float) :
        data_dict[key] = [ np.float64(k) for k in data_dict[key]]
    elif isinstance(data_dict[key][0],int) :
        data_dict[key] = [ np.int64(k) for k in data_dict[key]]

#For labelling in the plot we convert the genre and playlist tags to categorical variables:
data_dict , all_genres = analytics.make_genres_categorical(data_dict,return_genre_list=True)
data_dict , all_playlists = analytics.make_playlists_categorical(data_dict,return_playlist_list=True) 

#Convert the dictionary back to a dataframe for plotting with seaborn:
df_with_pca = pd.DataFrame.from_dict(data_dict)

In [5]:
#We take the most commonly occuring genres as there are many which only occur few times:
selected_genres = analytics.get_top_genres(data_dict,all_genres)

In [6]:
features = [ 'energy', 'liveness', 'tempo', 'speechiness', 'acousticness', 'instrumentalness', 'time_signature', 'danceability', 'key',  'duration_ms', 'loudness', 'valence', 'mode']
colour_choice = [ "genre" , "feature" , "playlist"]
playlists = all_playlists

plt.rcParams["figure.figsize"] = [ 10 ,10]
fig = plt.figure()

#replace w with list of most common genres:
def update(colour_select = colour_choice,genre = selected_genres,feature_size=features ,feature_colour=features,playlist=playlists  ):
    
    if colour_select == "genre" :
        ax = sns.scatterplot(x="PCA_X", y="PCA_Y", data=df_with_pca,hue=genre,size=feature_size,sizes=(50,300))
    
    elif colour_select == "feature" : 
        ax = sns.scatterplot(x="PCA_X", y="PCA_Y", data=df_with_pca,hue=feature_colour,size=feature_size,sizes=(50,300))
    
    elif colour_select == "playlist" : 
        ax = sns.scatterplot(x="PCA_X", y="PCA_Y", data=df_with_pca,hue=playlist,size=feature_size,sizes=(50,300))
    
    
    plt.legend(loc='upper right')
    fig.canvas.draw()

interact(update);

<Figure size 720x720 with 0 Axes>

interactive(children=(Dropdown(description='colour_select', options=('genre', 'feature', 'playlist'), value='g…

# Plot features on x and y axis

In [7]:
fig = plt.figure()

def update(x_axis = features, y_axis=features,colour_select = colour_choice,genre = selected_genres ,feature_colour=features,playlist=playlists , feature_size=features  ):
    
    if colour_select == "genre" :
        ax = sns.scatterplot(x=x_axis, y=y_axis, data=df_with_pca,hue=genre,size=feature_size,sizes=(50,300))
    elif colour_select == "feature" :
        x = sns.scatterplot(x=x_axis, y=y_axis, data=df_with_pca,hue=feature_colour,size=feature_size,sizes=(50,300))
    
    elif colour_select == "playlist" : 
        ax = sns.scatterplot(x=x_axis, y=y_axis, data=df_with_pca,hue=playlist,size=feature_size,sizes=(50,300))
    
    
    plt.legend(loc='upper right')
    fig.canvas.draw()

    

interact(update);

<Figure size 720x720 with 0 Axes>

interactive(children=(Dropdown(description='x_axis', options=('energy', 'liveness', 'tempo', 'speechiness', 'a…